## Backtracking solution.

In [4]:
import numpy as np
from sudoku import Sudoku
from itertools import product

In [255]:
def check_sudoku(grid):
    for i in range(3):
        for j in range(3):
            s = [
                grid[i*3][j*3],
                grid[i*3][j*3+1],
                grid[i*3][j*3+2],
                grid[i*3+1][j*3],
                grid[i*3+1][j*3+1],
                grid[i*3+1][j*3+2],
                grid[i*3+2][j*3],
                grid[i*3+2][j*3+1],
                grid[i*3+2][j*3+2]
            ]
            if sum(set(s)) != sum(s):
                return False
    for i in range(9):
        if sum(set(grid[i])) != sum(grid[i]):
            return False
    for j in range(9):
        col = [grid[i][j] for i in range(9)]
        if sum(set(col)) != sum(col):
            return False
    return True

In [220]:
puzzle = Sudoku(3).difficulty(.5)
board = np.array(puzzle.board)

board[board == None] = 0

print(f'Initial Board:\n{board}')

Initial Board:
[[5 0 0 2 0 6 0 7 4]
 [2 0 0 7 8 4 0 6 0]
 [0 6 0 1 3 5 0 0 0]
 [0 5 0 0 0 0 9 0 0]
 [0 4 0 3 0 0 5 8 0]
 [0 7 1 8 5 9 2 4 0]
 [1 3 0 0 2 0 6 9 8]
 [0 0 0 0 1 0 4 5 3]
 [0 8 5 9 4 0 0 0 0]]


In [221]:
def is_unique(lst):
    return lst.shape[0] == np.unique(lst).shape[0]

def can_put_number_at(x, y, n):
    global board
    
    board[x][y] = n
    row = board[x]
    col = board[:, y]
    res = is_unique(row[row != 0]) and is_unique(col[col != 0])

    ranges = [(0,3), (3,6), (6,9)]
    for left, right in product(ranges, ranges):
        current = board[left[0]:left[1], right[0]:right[1]]
        current = current[current != 0]
        res &= is_unique(current)

    board[x][y] = 0
    return res

#can_put_number_at(4,4,3)

In [222]:
solution_count = 0
res_backtracking = []
def solve():
    global board, solution_count
    for x in range(9):
        for y in range(9):
            if board[x][y] != 0:
                continue
            for n in range(1,10):
                if can_put_number_at(x, y, n):
                    board[x][y] = n
                    solve()
                    # backtracking step (revert to state before recursive call).
                    board[x][y] = 0
            return
    
    solution_count += 1
    res_backtracking.append(board)
solve()

## Solution with SAT Solver

In [223]:
# example usage
from z3.z3 import *

x = z3.Int('x')
y = z3.Int('y')
z = z3.Int('z')

solver = z3.Solver()
solver.add(2 * x == 10)
solver.add(x * y == 15)
solver.add(x + y + z == 10)

solver.check()
solver.model()

[y = 3, z = 2, x = 5]

In [268]:
X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(9) ] 
      for i in range(9) ]

# each cell contains a value in {1, ..., 9}
cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9) 
             for i in range(9) for j in range(9) ]

# each row contains a digit at most once
rows_c   = [ Distinct(X[i]) for i in range(9) ]

# each column contains a digit at most once
cols_c   = [ Distinct([ X[i][j] for i in range(9) ]) 
             for j in range(9) ]

# each 3x3 square contains a digit at most once
sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j] 
                        for i in range(3) for j in range(3) ]) 
             for i0 in range(3) for j0 in range(3) ]

sudoku_c = cells_c + rows_c + cols_c + sq_c

# sudoku instance, we use '0' for empty cells
instance = ((5,3,0,0,7,0,0,0,0),
            (6,0,0,1,9,5,0,0,0),
            (0,9,8,0,0,0,0,6,0),
            (8,0,0,0,6,0,0,0,3),
            (4,0,0,8,0,3,0,0,1),
            (7,0,0,0,2,0,0,0,6),
            (0,6,0,0,0,0,2,8,0),
            (0,0,0,4,1,9,0,0,5),
            (0,0,0,0,8,0,0,7,9))

instance_c = [ If(instance[i][j] == 0, 
                  True, 
                  X[i][j] == instance[i][j]) 
               for i in range(9) for j in range(9) ]

s = Solver()
s.add(sudoku_c + instance_c)
if s.check() == sat:
    m = s.model()
    res_sat_solver = [ [ m.evaluate(X[i][j]) for j in range(9) ] 
          for i in range(9) ]
    print_matrix(res_sat_solver)
else:
    print('failed to solve puzzle')

[[5, 3, 4, 6, 7, 8, 9, 1, 2],
 [6, 7, 2, 1, 9, 5, 3, 4, 8],
 [1, 9, 8, 3, 4, 2, 5, 6, 7],
 [8, 5, 9, 7, 6, 1, 4, 2, 3],
 [4, 2, 6, 8, 5, 3, 7, 9, 1],
 [7, 1, 3, 9, 2, 4, 8, 5, 6],
 [9, 6, 1, 5, 3, 7, 2, 8, 4],
 [2, 8, 7, 4, 1, 9, 6, 3, 5],
 [3, 4, 5, 2, 8, 6, 1, 7, 9]]


In [270]:
sat_to_ints = [[5, 3, 4, 6, 7, 8, 9, 1, 2],
 [6, 7, 2, 1, 9, 5, 3, 4, 8],
 [1, 9, 8, 3, 4, 2, 5, 6, 7],
 [8, 5, 9, 7, 6, 1, 4, 2, 3],
 [4, 2, 6, 8, 5, 3, 7, 9, 1],
 [7, 1, 3, 9, 2, 4, 8, 5, 6],
 [9, 6, 1, 5, 3, 7, 2, 8, 4],
 [2, 8, 7, 4, 1, 9, 6, 3, 5],
 [3, 4, 5, 2, 8, 6, 1, 7, 9]]

In [271]:
puzzle.solve().show_full()

# validate solutions
for rb in res_backtracking:
    print(check_sudoku(rb))
print(check_sudoku(sat_to_ints))


---------------------------
9x9 (3x3) SUDOKU PUZZLE
Difficulty: SOLVED
---------------------------
+-------+-------+-------+
| 5 1 8 | 2 9 6 | 3 7 4 |
| 2 9 3 | 7 8 4 | 1 6 5 |
| 4 6 7 | 1 3 5 | 8 2 9 |
+-------+-------+-------+
| 8 5 2 | 4 6 1 | 9 3 7 |
| 9 4 6 | 3 7 2 | 5 8 1 |
| 3 7 1 | 8 5 9 | 2 4 6 |
+-------+-------+-------+
| 1 3 4 | 5 2 7 | 6 9 8 |
| 7 2 9 | 6 1 8 | 4 5 3 |
| 6 8 5 | 9 4 3 | 7 1 2 |
+-------+-------+-------+

        
True
True
True


## SAT solution with Numpy

In [10]:
from z3.z3 import *

puzzle = Sudoku(3).difficulty(.5)
board = np.array(puzzle.board)

board[board == None] = 0

print(f'Initial Board:\n{board}')

Initial Board:
[[4 7 0 0 2 0 8 0 0]
 [0 0 8 4 3 0 5 9 0]
 [2 0 3 8 1 9 4 6 7]
 [0 8 0 9 0 3 0 4 1]
 [9 0 1 0 0 0 3 8 5]
 [0 0 0 0 4 8 0 7 0]
 [0 0 5 7 0 4 0 0 0]
 [0 9 0 0 0 0 0 0 0]
 [1 6 0 3 8 0 9 2 4]]


In [22]:
board = np.array([
    [Int(f'x_{i}_{j}') for j in range(9)]
    for i in range(9)
])

In [33]:
board[0:3,0:3]

array([[x_0_0, x_0_1, x_0_2],
       [x_1_0, x_1_1, x_1_2],
       [x_2_0, x_2_1, x_2_2]], dtype=object)

In [38]:
# we need to do BFS/DFS to search through a potential solution's grid
# and figure out if we can -> visit ALL numbered cells in one go (one connected region)